Import the neccessary libraries

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score,accuracy_score

Set visualiation style

In [12]:
sns.set_style("whitegrid")

Data Loading

In [13]:
DATASET_PATH = "database.csv"
try:
    df = pd.read_csv("database.csv")
except Exception as e:
    print(f"Error loading CSV: {e}")


df

,Player,Team,#,Nation,Position,Age,Minutes,Goals,Assists,Penalty Shoot on Goal,Penalty Shoot,Total Shoot,Shoot on Target,Yellow Cards,Red Cards,Touches,Dribbles,Tackles,Blocks,Expected Goals (xG),Non-Penalty xG (npxG),Expected Assists (xAG),Shot-Creating Actions,Goal-Creating Actions,Passes Completed,Passes Attempted,Pass Completion %,Progressive Passes,Carries,Progressive Carries,Dribble Attempts,Successful Dribbles,Date
0,Gorka Guruzeta,Athletic Club,12,ESP,FW,27-338,90,0,1,0,0,1,0,0,0,30,0,0,3,0.1,0.1,0.1,2,1,16,21,"76,2",0,15,0,1,0,2024-08-15
1,Álex Berenguer,Athletic Club,7,ESP,LW,29-042,71,0,0,0,0,0,0,0,0,33,1,0,1,0.0,0.0,0.0,0,0,12,23,"52,2",1,17,3,5,2,2024-08-15
2,Nico Williams,Athletic Club,10,ESP,LW,22-034,19,0,0,0,0,2,2,0,0,11,0,0,0,0.1,0.1,0.0,1,0,4,8,"50,0",0,6,2,1,1,2024-08-15
3,Iñaki Williams,Athletic Club,9,GHA,RW,30-061,90,0,0,0,0,2,0,0,0,36,1,0,0,0.1,0.1,0.0,0,0,17,26,"65,4",1,22,1,2,1,2024-08-15
4,Oihan Sancet,Athletic Club,8,ESP,AM,24-112,90,1,0,0,0,1,1,0,0,26,1,0,0,0.0,0.0,0.0,0,0,18,23,"78,3",3,15,2,0,0,2024-08-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4653,Antonio Rüdiger,Real Madrid,22,GER,CB,31-276,90,0,0,0,0,2,0,0,0,105,1,0,1,0.1,0.1,0.0,2,0,83,89,"93,3",5,65,1,0,0,2024-12-04
4654,Raúl Asencio,Real Madrid,35,ESP,CB,21-295,90,0,0,0,0,0,0,0,0,86,2,2,0,0.0,0.0,0.0,1,0,71,75,"94,7",1,52,0,1,1,2024-12-04
4655,Lucas Vázquez,Real Madrid,17,ESP,RB,33-156,87,0,0,0,0,0,0,1,0,60,3,0,1,0.0,0.0,0.0,0,0,44,51,"86,3",4,33,0,0,0,2024-12-04
4656,Arda Güler,Real Madrid,15,TUR,CM,19-283,3,0,0,0,0,0,0,0,0,13,0,0,0,0.0,0.0,0.0,0,0,9,11,"81,8",0,7,0,0,0,2024-12-04


In [14]:
# Define a function to clean numerical columns that use commas as decimal separators
def clean_numeric_columns(series):
    # Check if the series is of object type (string)
    if series.dtype == "object":
        # Replace the comma decimal separator with a dot and convert to float
        return series.str.replace(",",".",regex=True).astype(float)
    return series